**Diplomado en Inteligencia Artificial y Aprendizaje Profundo**

# Modelo de Regresión multi-logística con tf.keras 2.xx

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Importa módulos](#Importa-módulos)
* [Funciones de activación](#Funciones-de-activación)
* [El conjunto de datos Iris](#El-conjunto-de-datos-Iris)
* [Lectura de datos](#Lectura-de-datos)
* [Preprocesamiento](#Preprocesamiento)
* [Crea el modelo usando la API funcional](#Crea-el-modelo-usando-la-API-funcional)
* [Compila](#Compila)
* [Entrena](#Entrena)
* [Evaluación del modelo](#Evaluación-del-modelo)
* [Predicciones](#Predicciones)
* [Matriz de confusión](#Matriz-de-confusión)



## Introducción

Eeta lección está dedicada a un modelo  de clasificación con mútliples categoŕias, que corresponde a la generalización natural del modelo logístico.

Practicaremos la codificación *one-hot* para los datos de salida.

También introducimos la API funcional de tf.keras, que es una forma de programación maś flexible y poderosa.

Usaremos las funciones *relu* para capas intermedias y entrada y la función de activación *softmax* para la salida, debido a que se tienen varias clases. 

Adicionalmente, 

## Importa módulos

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
#
#from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
#
from tensorflow.keras.layers import Dense, Input, Activation
#
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
#
#from sklearn import KFold
print(tf.__version__)

## Funciones de activación

### Relu

Dada la salida del sumador digamos $y=\mathbf{w}'\mathbf{x} +b$, la función de activación *relu* esta definida por

$$
\text{relu}(y) = \begin{cases} &0, \text{ si } y\le 0,\\
 &y, \text{ en otro caso } \end{cases}
$$



### Softmax


Dados los valores $x_1,\ldots, x_n$ la función *softmax *  es definida por

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}
$$

Es decir, *softmax* transforma los valores en un función de probabilidad.

## El conjunto de datos Iris


Este conjunto de datos fue introducido por sir [Ronald Fisher]()

## Lectura de datos

In [ ]:
# nombres de las columnas de los datos
col_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
target_dimensions = ['Setosa', 'Versicolor', 'Virginica']

# lee los datos
training_data_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_data_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

training = pd.read_csv(training_data_path, names=col_names, header=0)
test = pd.read_csv(test_data_path, names=col_names, header=0)

## Pre-procesamiento

La variable objetivo (target) tiene tres categorías. Usaremos la codificación one-hot.

### Codificación one-hot

In [ ]:
y_train= pd.DataFrame(to_categorical(training.Species))
y_train.columns = target_dimensions

y_test = pd.DataFrame(to_categorical(test.Species))
y_test.columns = target_dimensions

In [ ]:
y_test

### Elimina columna Species 

In [ ]:
training.drop(['Species'], axis=1, inplace=True)
#test.drop(['Species'], axis=1, inplace=True)
y_test_species = test.pop('Species') # extrae la columna y la coloca en y_test_species
#
#Si necesita subir al dataframe la recodificación use estas líneas
#training = training.join(labels_training )
#test = test.join(labels_test )

### Normaliza los features

#### StandardScaler

In [ ]:
# crea el objeto StandardScaler
scaler = StandardScaler()

# Ajusta los parámetros del scaler
scaler.fit(training)
print (scaler.mean_)

# escala training y test
x_train = scaler.transform(training)
x_test = scaler.transform(test)

# labels ( no requieren escalación)


## Crea el modelo usando la API funcional

La API funcional de Keras es bastante más flexible y poderosa que el modelo Sequential

In [ ]:
# Con la API funcion se requiere la capa Input que transforma la entrada 
# en un tensor de tensorflow directamente
#
inputs = Input(shape=(4,),name='capa_entrada')
#
# vamos construyendo capa por capa
x = Activation('relu')(inputs)
x = Dense(5, activation='relu',name='primera_capa_oculta')(x)
x = Dense(10, activation='relu', name='segunda_capa_oculta')(x)
outputs = Dense(3, activation='softmax', name='capa_salida')(x)

# Creamos ahora el modelo
model_iris = Model(inputs=inputs, outputs=outputs)

model_iris.summary()
plot_model(model_iris, to_file='../Imagenes/iris_model.png', 
           show_shapes=True)

## Compila

In [ ]:
model_iris.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Entrena

In [ ]:
history = model_iris.fit(x_train, y_train,
                    batch_size= 32,
                    epochs= 150,
                    validation_split=0.1
                   )

## Evaluación del modelo

In [ ]:

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Entrenamiento y validación '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_metric(history, 'loss')

In [ ]:
plot_metric(history, 'accuracy')

In [ ]:
model_iris.evaluate(x = x_test,y = y_test)

## Predicciones

In [ ]:
# Predicting the Test set results
y_pred = model_iris.predict(x_test)
y_pred_c = np.argmax(y_pred, axis=1)

## Matriz de confusión

In [ ]:
cm = confusion_matrix(y_test_species, y_pred_c)

In [ ]:
print("Our accuracy is {}%".format(((cm[0][0] + cm[1][1]+ cm[2][2])/y_test_species.shape[0])*100))

In [ ]:
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

[Regresar al inicio](#Contenido)